# Engenharia de Prompt

## 1. Conexão com o modelo

In [1]:
# Libs

from google.colab import userdata
from huggingface_hub import login
from huggingface_hub import InferenceClient
from IPython.display import Markdown, display

In [2]:
# Login Hugging Face e conexão com modelo

API_KEY = userdata.get('HF_TOKEN')
login(API_KEY)

MODEL = "meta-llama/Meta-Llama-3-8B-Instruct"
client = InferenceClient(
    model=MODEL
)

## 2. Funções auxiliares

In [3]:
# Função de geração de texto

def printmd(string):
    display(Markdown(string))

def chat_generate(
    messages,
    temperature=0.2,
    top_p=0.95,
    max_tokens=500
):
    response = client.chat.completions.create(
        messages=messages,
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
    )

    return response.choices[0].message.content

## 3. Configurações de Amostragem

In [4]:
prompt1 = [
    {
        "role": "user",
        "content": """Aqui estão as notas brutas de um fiscal sobre o 'Festival
        de Inverno' na Praça da Matriz: [Notas: Público estimado de 2.000
        pessoas; som ultrapassou 80 decibéis após as 22h; 5 barracas de comida
        sem alvará sanitário; presença massiva de famílias; decorações feitas
        por artesãos locais; nenhum incidente de segurança grave registrado].

        Tarefa: Com base nessas notas, escreva um texto de um parágrafo sobre
        o evento."""
    }
]

### 3.1. Cenário 1: Coerente e formal (mais estabilidade)

In [5]:
temperatura = 0.0
top_p = 0.1

for i in range(3):
  printmd(f"\n## RESPOSTA {i+1}:\n")
  printmd(chat_generate(
      prompt1,
      temperature = temperatura,
      top_p = top_p
    )
  )


## RESPOSTA 1:


O Festival de Inverno na Praça da Matriz foi um sucesso de público e organização, com uma estimativa de 2.000 pessoas presentes. Embora o som tenha ultrapassado os 80 decibéis após as 22h, o evento foi marcado por uma atmosfera alegre e familiar, com a presença massiva de famílias. Além disso, as decorações foram feitas por artesãos locais, adicionando um toque autêntico ao evento. No entanto, foi necessário um ajuste no volume do som para evitar perturbações. Infelizmente, foram encontradas 5 barracas de comida sem alvará sanitário, o que é um problema de saúde pública que precisa ser abordado. Apesar disso, o evento foi marcado por uma ausência de incidentes de segurança graves, o que é um resultado positivo.


## RESPOSTA 2:


O Festival de Inverno na Praça da Matriz foi um sucesso de público, com uma estimativa de 2.000 pessoas presentes. Embora o som tenha ultrapassado os 80 decibéis após as 22h, o evento foi marcado por uma atmosfera alegre e familiar, com a presença massiva de famílias. Além disso, as decorações foram feitas por artesãos locais, adicionando um toque autêntico ao evento. No entanto, houve uma falha na parte sanitária, com 5 barracas de comida sem alvará, o que pode ter gerado preocupações de saúde pública. Felizmente, não houve incidentes de segurança graves registrados, o que é um sinal positivo para o sucesso do evento.


## RESPOSTA 3:


O Festival de Inverno na Praça da Matriz foi um sucesso de público e organização, com uma estimativa de 2.000 pessoas presentes. Embora o som tenha ultrapassado os 80 decibéis após as 22h, o evento foi marcado por uma atmosfera alegre e familiar, com a presença massiva de famílias. Além disso, as decorações foram criadas por artesãos locais, adicionando um toque autêntico ao evento. No entanto, foi necessário um ajuste no volume do som para evitar problemas de ruído. Infelizmente, foram encontradas 5 barracas de comida sem alvará sanitário, o que é um problema de saúde pública que precisa ser resolvido. Apesar disso, o evento foi considerado seguro, sem incidentes graves registrados.

### 3.2. Cenário 2: Criativo (mais diversidade)

In [6]:
temperatura = 1.0
top_p = 1.0

for i in range(3):
  printmd(f"\n## RESPOSTA {i+1}:\n")
  printmd(chat_generate(
      prompt1,
      temperature = temperatura,
      top_p = top_p
    )
  )


## RESPOSTA 1:


O Festival de Inverno na Praça da Matriz foi um sucesso de público e diversão, tendo recebido uma estimativa de 2.000 pessoas. Embora o som tenha ultrapassado a recomendada faixa de 80 decibéis após as 22h, a atmosfera alegre e saturada de decorações criadas por artesãos locais contribuiu para a sensação de festa. A presença de famílias em massa é sinal de que o evento alcançou a sua principal audiência, além disso, ficou evidente a qualidade dos materiais consumidos através da comida servida. No entanto, é fundamental destacar que o fiscal identificou cinco barracas de comida operando sem alvará sanitário, um aspecto que deve ser cuidado para garantir a segurança dos frequentadores.


## RESPOSTA 2:


O "Festival de Inverno" na Praça da Matriz foi um sucesso, atraindo um público estimado de 2.000 pessoas. Embora as decorações tenham sido feitas de forma efetiva por artesãos locais, houve alguns problemas com as barracas de comida, que não tinham alvará sanitário, apesar de não haver registros de incidentes de segurança graves. Além disso, o som das apresentações foi um pouco intenso, ao ultrapassar os 80 decibéis após as 22h, mas a presença massiva de famílias na festa foi um grande atrativo, fazendo do evento um momento memorable para todos que participaram.


## RESPOSTA 3:


O Festival de Inverno na Praça da Matriz foi um evento de grande sucesso, com um público estimado de 2.000 pessoas. Embora houvesse um som que ultrapassou os 80 decibéis após as 22h, o evento foi marcado por uma atmosfera alegre e familiar, com a presença massiva de famílias que aproveitaram as decorações feitas por artesãos locais. Além disso, as organizações do evento também foram elogiadas por tomar medidas para garantir a segurança dos espectadores, o que foi reforçado pelo fato de não haver incidentes graves registrados durante o evento. Infelizmente, foi possível observar que as barracas de comida não estavam em conformidade com os padrões de saúde, faltando alvará sanitário, então há oportunidades de melhoria.

## 4. Arquitetura de Prompt

### 4.1. Cenário 1 (Fiscalização)

In [7]:
# System Prompt (Propósito e Formato)
system_prompt2 = '''
PROPÓSITO E FORMATO: Você é um sistema especializado em processamento de dados
administrativos e conformidade regulatória. Sua função é transformar anotações
de campo desestruturadas em relatórios técnicos precisos, garantindo que nenhum
dado factual seja omitido ou alterado. *Sua saída deve ser estritamente em
formato JSON válido*. Não inclua informações fora do JSON.

*Esquema JSON obrigatório:*

{
  "evento": string,
  "publico_alvo": string,
  "volume_estimado": int,
  "infracoes_detectadas": [
    {
      "tipo": string,
      "descricao": string,
      "risco_regulatorio": string
    }
  ],
  "balanco_seguranca": string
}
'''

# Role Prompt (Persona e Tom):
role_prompt2 = '''
PERSONA: Aja como um *Auditor de Serviços Públicos sênior*. Adote um tom
formal, imparcial e analítico. Sua expertise deve focar em identificar riscos
sanitários, conformidade sonora e segurança pública, priorizando a clareza
quanto ao impacto regulatório.
'''

# Contextual Prompt (Dados Variáveis)
context_prompt2 = '''
CONTEXTO: Fiscalização realizada no 'Festival de Inverno' na Praça da Matriz.

NOTAS BRUTAS: {Público estimado de 2.000 pessoas; som ultrapassou 80 decibéis
após as 22h; 5 barracas de comida sem alvará sanitário; presença massiva de
famílias; decorações feitas por artesãos locais; nenhum incidente de segurança
grave registrado}.
'''

# Tarefa e instruções
task_instructions2 = '''
TAREFA: *Analise* as notas acima e *gere* um resumo técnico estruturado em JSON.

*INSTRUÇÕES:* Liste as métricas de público, categorize as infrações por tipo
(sanitária/sonora) e descreva o impacto social positivo observado.
'''

In [8]:
prompt2 = [
    {
        "role": "system",
        "content": f"""
{system_prompt2}

{role_prompt2}

"""
    },
    {
        "role": "user",
        "content": f"""
{context_prompt2}

{task_instructions2}

Agora execute a tarefa.
"""
    }
]

In [9]:
temperatura = 0.1
top_p = 0.9

for i in range(2):
  printmd(f"\n## RESPOSTA {i+1}:\n")
  printmd(chat_generate(
      prompt2,
      temperature = temperatura,
      top_p = top_p
    )
  )


## RESPOSTA 1:


**Resumo Técnico do Fiscalização do Festival de Inverno**

```json
{
  "evento": "Festival de Inverno",
  "publico_alvo": "Praça da Matriz",
  "volume_estimado": 2000,
  "infracoes_detectadas": [
    {
      "tipo": "sanitária",
      "descricao": "5 barracas de comida sem alvará sanitário",
      "risco_regulatorio": "Risco de contaminação alimentar"
    },
    {
      "tipo": "sonora",
      "descricao": "Som ultrapassou 80 decibéis após as 22h",
      "risco_regulatorio": "Risco de danos auditivos"
    }
  ],
  "balanco_seguranca": "Nenhum incidente de segurança grave registrado"
}
```

**Análise e Observações**

* O público estimado foi de 2.000 pessoas, o que indica uma grande concentração de pessoas na área.
* Foram detectadas 2 infrações: 1 sanitária e 1 sonora.
 * A infração sanitária foi causada pela presença de 5 barracas de comida sem alvará sanitário, o que pode colocar em risco a saúde dos consumidores.
 * A infração sonora foi causada pelo som que ultrapassou 80 decibéis após as 22h, o que pode causar danos auditivos.
* A presença massiva de famílias foi observada, o que indica um impacto social positivo.
* As decorações feitas por artesãos locais também foram observadas, o que pode contribuir para a economia local e a preservação da cultura local.
* Nenhum incidente de segurança grave foi registrado, o que indica que as medidas de segurança foram eficazes.


## RESPOSTA 2:


**Resumo Técnico em JSON**

```json
{
  "evento": "Festival de Inverno na Praça da Matriz",
  "publico_alvo": "2.000 pessoas",
  "volume_estimado": 2000,
  "infracoes_detectadas": [
    {
      "tipo": "Sanitária",
      "descricao": "5 barracas de comida sem alvará sanitário",
      "risco_regulatorio": "Risco de contaminação alimentar"
    },
    {
      "tipo": "Sonora",
      "descricao": "Som ultrapassou 80 decibéis após as 22h",
      "risco_regulatorio": "Risco de danos auditivos"
    }
  ],
  "balanco_seguranca": "Nenhum incidente de segurança grave registrado"
}
```

**Análise e Observações**

*   O público estimado foi de 2.000 pessoas, o que indica uma grande concentração de pessoas na área.
*   As infrações detectadas foram categorizadas em sanitária e sonora.
*   A presença de 5 barracas de comida sem alvará sanitário representa um risco significativo de contaminação alimentar.
*   O som ultrapassando 80 decibéis após as 22h representa um risco de danos auditivos.
*   A presença massiva de famílias no evento sugere um impacto social positivo, pois o festival parece ter sido uma experiência agradável e segura para as famílias.
*   A decoração feita por artesãos locais também sugere um impacto positivo na economia local e na cultura da comunidade.

### 4.2. Cenário 2 (Marketing Institucional)

In [10]:
# System Prompt (Propósito e Formato)
system_prompt3 = '''
PROPÓSITO E FORMATO: Você é um especialista em *Comunicação Social e Marketing
Governamental*. Sua função é transformar dados brutos de fiscalização em
eventos em *narrativas institucionais inspiradoras* que valorizem a imagem da
prefeitura e o bem-estar da comunidade. Você deve garantir que a saída seja
rica em detalhes, mas extremamente concisa. *Sua saída deve ser estritamente em
formato JSON válido* para  ser consumida por plataformas de redes sociais e
sites oficiais. Não inclua explicações fora do JSON.

*Esquema JSON obrigatório:*
{
  "manchete_impacto": string,
  "corpo_texto_denso": string,
  "destaque_comunitario": string,
  "hashtags_sugeridas": string
}
'''

# Role Prompt (Persona e Tom):
role_prompt3 = '''
PERSONA: Aja como um *Redator Publicitário Criativo*. Adote um tom *Inspirador,
Persuasivo e Influente*. Sua expertise deve focar em destacar o sucesso do
evento, a segurança das famílias e o apoio à economia local, transformando
números em conquistas sociais.
'''

# Contextual Prompt (Dados Variáveis)
context_prompt3 = '''
CONTEXTO: Fiscalização realizada no 'Festival de Inverno' na Praça da Matriz.

NOTAS BRUTAS: {Público estimado de 2.000 pessoas; som ultrapassou 80 decibéis
após as 22h; 5 barracas de comida sem alvará sanitário; presença massiva de
famílias; decorações feitas por artesãos locais; nenhum incidente de segurança
grave registrado}.
'''

# Tarefa e instruções
task_instructions3 = '''
TAREFA: Crie uma síntese narrativa do evento.

INSTRUÇÕES: Foque na harmonia e no sucesso do evento; Use verbos de ação como
'Celebrar', 'Fortalecer' e 'Encantar'.
'''

In [11]:
prompt3 = [
    {
        "role": "system",
        "content": f"""
{system_prompt3}

{role_prompt3}

"""
    },
    {
        "role": "user",
        "content": f"""
{context_prompt3}

{task_instructions3}

Agora execute a tarefa.
"""
    }
]

In [12]:
temperatura = 0.9
top_p = 0.99

for i in range(2):
  printmd(f"\n## RESPOSTA {i+1}:\n")
  printmd(chat_generate(
      prompt3,
      temperature = temperatura,
      top_p = top_p
    )
  )


## RESPOSTA 1:


{
  "manchete_impacto": "Festival de Inverno: Uma Noite de Harmonia e Celebração!",
  "corpo_texto_denso": "Com uma afluência de 2.000 entusiastas, o Festival de Inverno na Praça da Matriz alcançou todos os níveis de sucesso. Mesmo após as 22h, o som vibrante alcançou 80 decibéis, sem perturbar a paz da noite. Além disso, as decorações feitas por habilidosos artesãos locais encantaram a todos que os viram. E, o que mais impressionou, foi a presença massiva de famílias, que celebraram a magia do inverno ao lado da prefeitura. É sempre emocionante ver uma cidade unida e forte, e o Festival de Inverno foi exemplo disso! A parceria entre os organizadores e o poder público fortaleceu a confiança da comunidade em nossa administração.",
  "destaque_comunitario": "Entidades locais e famílias desde casa apoiam e sonham com grandes eventos como o Festival de Inverno!",
  "hashtags_sugeridas": "#FestivalDeInverno #PraçaDaMatriz #CidadeUnida #FortalecerComunidade #DesenvolvimentoLocal"
}


## RESPOSTA 2:


{
  "manchete_impacto": "Festival de Inverno na Praça da Matriz: Harmonia e Sucesso em Maior Escala!",
  "corpo_texto_denso": "Nessa noite mágica, 2.000 pessoas se reuniram para celebrar a alegria e a beleza invernal. Além de ultrapassar os limites sonoros, o som encantou as multidões até as 22h. Confrontados a uma realidade local cada vez menos sustentável, a fiscalização também prestou atenção em barracas de comida locais que atuam na corregedoría. Fortalecendo a identidade local, realizamos um aberto mapeamento de suas demais atividades culinários. Ao fim, sem abalos de segurança e sem nenhum incidente, a noite foi encerrada com um grand finale",  
  "destaque_comunitario": "O Festival de Inverno não apenas atraíu famílias com decorações artesanais de bem qualidade, mas também os diversos lojistas locais com vendas surpreendentes em termos de faturamento.",
  "hashtags_sugeridas": "#FestivalDeInverno #PraçaDaMatriz #HarmoniaESegurança #EconomiaLocal #Artesanato #FamíliasUnidas"
}

## 5. Zero-shot, One-shot e Few-shot Prompting

Incremente o prompt com persona "Auditor de serviços públicos sênior" (zero-shot), com prompts one-shot e few-shot.

### 5.1. One-shot

In [13]:
# System Prompt (Propósito e Formato)
system_prompt4 = '''
PROPÓSITO E FORMATO: Você é um sistema especializado em processamento de dados
administrativos e conformidade regulatória. Sua função é transformar anotações
de campo desestruturadas em relatórios técnicos precisos, garantindo que nenhum
dado factual seja omitido ou alterado. *Sua saída deve ser estritamente em
formato JSON válido*. Não inclua informações fora do JSON.

REGRAS OBRIGATÓRIAS:
1. Responda exclusivamente com JSON válido.
2. Não inclua texto antes ou depois do JSON.
3. Não use markdown.
4. Não inclua comentários.
5. A resposta deve começar com { e terminar com }.
6. Não adicione chaves extras.
7. Não omita chaves.

*Esquema JSON obrigatório:*

{
  "evento": string,
  "publico_alvo": string,
  "volume_estimado": int,
  "infracoes_detectadas": [
    {
      "tipo": string,
      "descricao": string,
      "risco_regulatorio": string
    }
  ],
  "balanco_seguranca": string
}
'''

exemplo1 = """
*EXEMPLO:*

CONTEXTO: Fiscalização realizada na "Feira Noturna Municipal".

NOTAS BRUTAS: {Público estimado de 800 pessoas; música ambiente dentro do limite
legal de 60 decibéis; 2 barracas sem licença sanitária; presença predominante
de adultos; nenhuma ocorrência policial registrada}.

Saída esperada para o exemplo:

{
  "evento": "Feira Noturna Municipal",
  "publico_alvo": "Predominância de adultos",
  "volume_estimado": 800,
  "infracoes_detectadas": [
    {
      "tipo": "Sanitária",
      "descricao": "Duas barracas operando sem licença sanitária",
      "risco_regulatorio": "Potencial risco à saúde pública e descumprimento de
      normas municipais"
    }
  ],
  "balanco_seguranca": "Não houve ocorrências policiais ou incidentes graves."
}

--- FIM DO EXEMPLO ---
"""

In [14]:
prompt4 = [
    {
        "role": "system",
        "content": f"""
{system_prompt4}

{role_prompt2}

"""
    },
    {
        "role": "user",
        "content": f"""
{exemplo1}

Agora execute a tarefa real abaixo:

{context_prompt2}

{task_instructions2}
"""
    }
]

In [15]:
temperatura = 0.2
top_p = 0.9

printmd(chat_generate(
    prompt4,
    temperature = temperatura,
    top_p = top_p
  )
)

{
  "evento": "Festival de Inverno",
  "publico_alvo": "Famílias",
  "volume_estimado": 2000,
  "infracoes_detectadas": [
    {
      "tipo": "Sanitária",
      "descricao": "Cinco barracas de comida operando sem alvará sanitário",
      "risco_regulatorio": "Potencial risco à saúde pública e descumprimento de normas municipais"
    },
    {
      "tipo": "Sonora",
      "descricao": "Som ultrapassou 80 decibéis após as 22h",
      "risco_regulatorio": "Potencial risco à saúde auditiva e descumprimento de normas municipais"
    }
  ],
  "balanco_seguranca": "Não houve incidentes de segurança graves registrados e a presença de decorações feitas por artesãos locais teve um impacto social positivo na comunidade."
}

### 5.2. Few-shot

In [16]:
exemplo2 = '''
CONTEXTO: Fiscalização na "Feira Gastronômica Municipal".

NOTAS BRUTAS: {Público estimado de 1.200 pessoas; som ambiente dentro do limite
legal; 3 barracas sem licença sanitária; público majoritariamente adulto;
nenhuma ocorrência policial}.

SAÍDA:

{
  "evento": "Feira Gastronômica Municipal",
  "publico_alvo": "Predominância de adultos",
  "volume_estimado": 1200,
  "infracoes_detectadas": [
    {
      "tipo": "Sanitária",
      "descricao": "Três barracas operando sem licença sanitária",
      "risco_regulatorio": "Descumprimento das normas sanitárias municipais com
      potencial risco à saúde pública"
    }
  ],
  "balanco_seguranca": "Evento transcorreu sem ocorrências policiais ou
  incidentes graves."
}
'''

exemplo3 = '''
CONTEXTO: Fiscalização no "Show Cultural da Juventude".

NOTAS BRUTAS: {Público estimado de 3.500 pessoas; som atingiu 95 decibéis após
as 23h; todas as barracas com alvará regular; presença significativa de jovens;
registro de 1 atendimento médico leve}.

SAÍDA:

{
  "evento": "Show Cultural da Juventude",
  "publico_alvo": "Predominância de jovens",
  "volume_estimado": "Aproximadamente 3.500 participantes",
  "infracoes_detectadas": [
    {
      "tipo": "Sonora",
      "descricao": "Emissão sonora acima de 95 decibéis após as 23h",
      "risco_regulatorio": "Descumprimento da legislação municipal de controle
      de ruído"
    }
  ],
  "balanco_seguranca": "Evento com um atendimento médico leve e sem incidentes
  de segurança grave."
}
'''

In [17]:
prompt5 = [
    {
        "role": "system",
        "content": f"""
{system_prompt4}

{role_prompt2}

"""
    },
    {
        "role": "user",
        "content": f"""
EXEMPLO 1:
{exemplo2}

EXEMPLO 2:
{exemplo3}

Agora execute a tarefa real:

{context_prompt2}

{task_instructions2}
"""
    }
]

In [18]:
temperatura = 0.2
top_p = 0.9

printmd(chat_generate(
    prompt5,
    temperature = temperatura,
    top_p = top_p
  )
)

{
  "evento": "Festival de Inverno",
  "publico_alvo": "Famílias",
  "volume_estimado": 2000,
  "infracoes_detectadas": [
    {
      "tipo": "Sanitária",
      "descricao": "Cinco barracas de comida operando sem alvará sanitário",
      "risco_regulatorio": "Descumprimento das normas sanitárias municipais com 
      potencial risco à saúde pública"
    },
    {
      "tipo": "Sonora",
      "descricao": "Emissão sonora acima de 80 decibéis após as 22h",
      "risco_regulatorio": "Descumprimento da legislação municipal de controle 
      de ruído"
    }
  ],
  "balanco_seguranca": "Evento com presença massiva de famílias, decorações 
  feitas por artesãos locais e sem incidentes de segurança grave registrados."
}

## 6. Chain-of-Thought (CoT)

### 6.1. Prompt simples

In [19]:
prompt6 = messages = [
  {
    "role": "system",
    "content": """Você é um assistente técnico de auditoria fiscal tributária.
    Forneça respostas objetivas, numéricas e diretamente aplicáveis. Não
    explique o raciocínio. Apresente apenas o valor final solicitado."""
  },
  {
    "role": "user",
    "content": """Calcule o valor total devido (imposto + multa) para a Empresa
    Y com base nos dados abaixo:

Receita Bruta: R$ 1.450.000,00
Alíquota Progressiva:
- 2% sobre os primeiros R$ 500.000,00
- 5% sobre o valor excedente

Dedução permitida: R$ 120.000,00 (aplicada sobre o total do imposto calculado).
Multa: 1% ao mês (juros simples) sobre o valor líquido devido.
Período de atraso: 5 meses.

Retorne apenas:
Valor total a recolher: R$ X.XXX.XXX,XX
"""
  }
]

In [20]:
temperatura = 0.0
top_p = 1.0         # Irrelevante!

resposta = chat_generate(
    prompt6,
    temperature = temperatura,
    top_p = top_p
)

In [21]:
print(resposta)

**Calculo do Imposto**

- Valor sobre os primeiros R$ 500.000,00: R$ 500.000,00 * 2% = R$ 10.000,00
- Valor excedente: R$ 1.450.000,00 - R$ 500.000,00 = R$ 950.000,00
- Imposto sobre o valor excedente: R$ 950.000,00 * 5% = R$ 47.500,00
- Imposto total: R$ 10.000,00 + R$ 47.500,00 = R$ 57.500,00

**Calculo da Dedução**

- Dedução permitida: R$ 57.500,00 * (R$ 120.000,00 / R$ 57.500,00) = R$ 120.000,00 (aplicada sobre o total do imposto calculado)

**Calculo do Valor Líquido devido**

- Imposto total: R$ 57.500,00
- Dedução permitida: R$ 120.000,00
- Valor líquido devido: R$ 57.500,00 - R$ 120.000,00 = R$ -62.500,00 (não pode ser negativo, portanto, o valor líquido devido é R$ 0,00)

**Calculo da Multa**

- Valor líquido devido: R$ 0,00
- Período de atraso: 5 meses
- Multa: R$ 0,00 * 1% * 5 = R$ 0,00

**Calculo do Valor Total a Recolher**

- Imposto total: R$ 57.500,00
- Multa: R$ 0,00
- Valor total a recolher: R$ 57.500,00 + R$ 0,00 = R$ 57.500,00


### 6.2. Prompt com CoT

In [22]:
prompt7 = [
  {
    "role": "system",
    "content": """Você é um Auditor Fiscal Sênior especialista em cálculos
    tributários complexos.
Sua prioridade é precisão matemática absoluta.
Antes de apresentar o valor final, você deve estruturar explicitamente todas as
etapas de cálculo.
Apresente cada etapa numerada e com valores intermediários claros."""
  },
  {
    "role": "user",
    "content": """Realize a auditoria fiscal da Empresa Y.

DADOS:
- Receita Bruta: R$ 1.450.000,00
- Alíquota Progressiva:
  • 2% até R$ 500.000,00
  • 5% sobre o excedente
- Dedução: R$ 120.000,00 (aplicada após cálculo total do imposto)
- Juros: 1% ao mês (juros simples)
- Atraso: 5 meses

INSTRUÇÕES OBRIGATÓRIAS:

1. Separe explicitamente a receita nas duas faixas de tributação.
2. Calcule o imposto de cada faixa individualmente.
3. Some os impostos para obter o imposto bruto total.
4. Subtraia a dedução e identifique o valor líquido devido.
5. Calcule os juros simples acumulados (1% x 5 meses).
6. Some o valor líquido e os juros.
7. Revise o resultado final antes de apresentá-lo.

Apresente:
- Cada etapa numerada
- Valores intermediários
- Fórmulas utilizadas
- Resultado final destacado
"""
  }
]

In [23]:
temperatura = 0.0
top_p = 1.0         # Irrelevante!

resposta = chat_generate(
    prompt7,
    temperature = temperatura,
    top_p = top_p,
    max_tokens=1000
)

In [24]:
print(resposta)

**Auditoria Fiscal da Empresa Y**

**Etapa 1: Separação da Receita nas Duas Faixas de Tributação**

* Receita Bruta Total: R$ 1.450.000,00
* Faixa 1 (até R$ 500.000,00): R$ 500.000,00
* Faixa 2 (excedente): R$ 950.000,00 (R$ 1.450.000,00 - R$ 500.000,00)

**Etapa 2: Cálculo do Imposto de Cada Faixa Individualmente**

* Faixa 1 (2% até R$ 500.000,00):
 + Base de Cálculo: R$ 500.000,00
 + Alíquota: 2%
 + Imposto: R$ 500.000,00 x 0,02 = R$ 10.000,00
* Faixa 2 (5% sobre o excedente):
 + Base de Cálculo: R$ 950.000,00 (excedente)
 + Alíquota: 5%
 + Imposto: R$ 950.000,00 x 0,05 = R$ 47.500,00
* Imposto Bruto Total: R$ 10.000,00 + R$ 47.500,00 = R$ 57.500,00

**Etapa 3: Cálculo do Imposto Bruto Total**

* Imposto Bruto Total: R$ 57.500,00 (já calculado na Etapa 2)

**Etapa 4: Subtração da Dedução e Identificação do Valor Líquido devido**

* Imposto Bruto Total: R$ 57.500,00
* Dedução: R$ 120.000,00
* Valor Líquido devido: R$ 57.500,00 - R$ 120.000,00 = - R$ 62.500,00 (não pode ser negativo, 

## 7. Chain-of-Verification (CoV)

In [25]:
prompt8 = [
    {
        "role": "user",
        "content": """Cite alguns políticos que nasceram no estado de Pernambuco"""
    }
]

In [26]:
temperatura = 0.2
top_p = 0.9

resposta = chat_generate(
    prompt8,
    temperature = temperatura,
    top_p = top_p
)

In [27]:
printmd(resposta)

Aqui estão alguns políticos que nasceram no estado de Pernambuco:

1. **José Sarney**: Nascido em 1930, em São Bento do Una, Pernambuco, foi presidente do Brasil de 1985 a 1990 e senador por várias legislaturas.
2. **Marconi Perillo**: Nascido em 1960, em Recife, Pernambuco, é um político e empresário que foi governador de Goiás de 2003 a 2011.
3. **Eduardo Campos**: Nascido em 1962, em Recife, Pernambuco, foi um político e empresário que foi governador de Pernambuco de 2007 a 2014 e candidato à presidência da República em 2014.
4. **Armando Monteiro Neto**: Nascido em 1952, em Recife, Pernambuco, é um político e empresário que foi governador de Pernambuco de 2001 a 2006 e senador por várias legislaturas.
5. **João Paulo**: Nascido em 1957, em Recife, Pernambuco, é um político que foi governador de Pernambuco de 1991 a 1994 e senador por várias legislaturas.
6. **João Lyra Neto**: Nascido em 1953, em Recife, Pernambuco, é um político que foi governador de Pernambuco de 1995 a 1999.
7. **Miguel Arraes**: Nascido em 1922, em Aracaju, mas com raízes em Pernambuco, foi um político que foi governador de Pernambuco de 1961 a 1964 e senador por várias legislaturas.
8. **Ciro Gomes**: Nascido em 1955, em Fortaleza, mas com raízes em Pernambuco, é um político que foi governador do Ceará de 2003 a 2010 e candidato à presidência da República em 2018.

Essa é apenas uma lista parcial de políticos que nasceram em Pernambuco. Existem muitos outros que também

In [28]:
prompt9 = [
    {
        "role": "system",
        "content": """Você é um assistente especializado em verificação factual e precisão histórica.
Sempre que listar fatos, deve justificar e verificar cada afirmação antes de apresentar o resultado final."""
    },
    {
        "role": "user",
        "content": """
Cite alguns políticos que nasceram no estado de Pernambuco.

INSTRUÇÕES OBRIGATÓRIAS:

1. Primeiro, elabore uma lista preliminar de políticos brasileiros que você acredita terem nascido em Pernambuco.
2. Para cada nome <NOME> da lista, responda a seguinte pergunta: Em que estado nasceu <NOME>?
3. Em seguida, revise cada item da lista preliminar gerada no passo 1 de acordo com as respostas do passo 2
4. Gere uma lista final revisada contendo apenas os nomes confirmados.

Organize sua resposta nas seguintes seções:

  - SEÇÃO 1: Lista Inicial
  - SEÇÃO 2: Verificação Item a Item
  - SEÇÃO 3: Lista Final Confirmada
"""
    }
]

In [29]:
temperatura = 0.2
top_p = 0.9

resposta = chat_generate(
    prompt9,
    temperature = temperatura,
    top_p = top_p
)

In [30]:
printmd(resposta)

**SEÇÃO 1: Lista Inicial**

Aqui está uma lista de políticos brasileiros que eu acredito terem nascido em Pernambuco:

1. Fernando Collor de Mello
2. Miguel Arraes
3. Miguel Calmon
4. Fernando Lyra
5. Miguel Arraes Filho
6. Eduardo Campos
7. João Lyra Neto
8. João Lyra Filho
9. Fernando Lyra Filho
10. João Lyra Neto Filho

**SEÇÃO 2: Verificação Item a Item**

Aqui estão as respostas para cada item da lista inicial:

1. **Fernando Collor de Mello**: Nasceu em Rio Branco, Acre.
2. **Miguel Arraes**: Nasceu em Recife, Pernambuco.
3. **Miguel Calmon**: Não consegui encontrar informações sobre um político brasileiro com esse nome.
4. **Fernando Lyra**: Nasceu em Recife, Pernambuco.
5. **Miguel Arraes Filho**: Não consegui encontrar informações sobre um político brasileiro com esse nome.
6. **Eduardo Campos**: Nasceu em Recife, Pernambuco.
7. **João Lyra Neto**: Não consegui encontrar informações sobre um político brasileiro com esse nome.
8. **João Lyra Filho**: Não consegui encontrar informações sobre um político brasileiro com esse nome.
9. **Fernando Lyra Filho**: Não consegui encontrar informações sobre um político brasileiro com esse nome.
10. **João Lyra Neto Filho**: Não consegui encontrar informações sobre um político brasileiro com esse nome.

**SEÇÃO 3: Lista Final Confirmada**

Aqui está a lista final de políticos brasileiros que nasceram em Pernambuco:

1. Miguel Arraes
2. Fernando Lyra
3. Eduardo Campos

## 8. Chain-of-Density (CoD)

### 8.1. Sem CoD

In [31]:
prompt10 = [
    {
        "role": "system",
        "content": """
Você é um assistente especializado em sumarização jurídica de alta precisão.
"""
    },
    {
        "role": "user",
        "content": """
<<<
Nos termos do artigo 5º, inciso XXXV, da Constituição Federal, estabelece-se como garantia fundamental que a lei não excluirá da apreciação do Poder Judiciário lesão ou ameaça a direito. Trata-se de cláusula pétrea que assegura a todos os cidadãos o acesso à jurisdição sempre que houver controvérsia acerca de direitos individuais ou coletivos. Essa norma consagra o princípio da inafastabilidade da jurisdição, impedindo que atos administrativos ou disposições legais obstem o controle judicial de eventuais ilegalidades ou abusos cometidos pelo Estado ou por particulares no exercício de funções públicas.

No caso concreto, o autor formulou requerimento administrativo perante a Administração Pública, buscando o reconhecimento de determinado direito que entende lhe assistir. Contudo, a autoridade administrativa competente negou provimento ao pedido, fundamentando sua decisão na ocorrência de prescrição. Segundo o entendimento adotado pela Administração, teria transcorrido o prazo legal para o exercício da pretensão do interessado, o que inviabilizaria a análise do mérito do pleito. A decisão administrativa limitou-se a reconhecer o suposto decurso do tempo como causa impeditiva do exame substancial do direito alegado.

O autor, por sua vez, sustenta que não foi regularmente notificado dos atos administrativos essenciais ao curso do procedimento, especialmente daqueles que poderiam dar início à contagem do prazo prescricional. Argumenta que a ausência de notificação válida comprometeu sua ciência inequívoca dos fatos e dos fundamentos utilizados pela Administração, configurando cerceamento de defesa. Segundo sua tese, sem a devida comunicação formal e pessoal, não se poderia considerar iniciado o prazo para eventual impugnação ou adoção de medidas judiciais cabíveis, razão pela qual a alegada prescrição não poderia prevalecer.

Ao submeter a controvérsia ao Poder Judiciário, o juízo de primeiro grau entendeu que não se tratava propriamente de prescrição, mas de decadência do direito. Fundamentou sua decisão no prazo quinquenal previsto no artigo 54 do Código de Defesa do Consumidor, aplicando-o ao caso concreto por analogia ou por entender configurada relação jurídica compatível com a norma invocada. Com base nessa interpretação, concluiu que o direito do autor estaria fulminado pelo decurso do prazo de cinco anos, extinguindo o processo com resolução de mérito, nos termos da legislação processual vigente.

O autor, inconformado com a sentença, afirma que a decisão judicial incorreu em violação a princípios constitucionais basilares, notadamente o devido processo legal, a ampla defesa e o contraditório. Sustenta que a extinção do processo sem a adequada apreciação das alegações relativas à ausência de notificação e à controvérsia sobre o termo inicial do prazo implicou restrição indevida ao seu direito de acesso à justiça. Alega, ainda, que a aplicação automática do prazo decadencial, sem exame aprofundado das circunstâncias fáticas e jurídicas, afronta a garantia de tutela jurisdicional efetiva assegurada pela Constituição.
>>>

Resuma o texto jurídico acima em até 150 palavras.
"""
    }
]

In [32]:
temperatura = 0.1
top_p = 0.9

resposta = chat_generate(
    prompt10,
    temperature = temperatura,
    top_p = top_p
)

In [33]:
printmd(resposta)

O autor formulou requerimento administrativo para reconhecimento de direito, mas a autoridade administrativa negou, fundamentando em prescrição. O autor alega que não foi regularmente notificado, o que comprometeu sua ciência inequívoca dos fatos e fundamentos utilizados pela Administração. O juízo de primeiro grau entendeu que não se tratava de prescrição, mas de decadência do direito, aplicando o prazo quinquenal do artigo 54 do Código de Defesa do Consumidor. A decisão extinguiu o processo com resolução de mérito.

O autor alega violação a princípios constitucionais basilares, como o devido processo legal, a ampla defesa e o contraditório. Sustenta que a extinção do processo sem a adequada apreciação das alegações e a aplicação automática do prazo decadencial afrontam a garantia de tutela jurisdicional efetiva. O caso envolve a aplicação do princípio da inafastabilidade da jurisdição e a garantia de acesso à justiça.

### 8.2. Com CoD

In [34]:
prompt11 = [
    {
        "role": "system",
        "content": """
Você é um assistente especializado em sumarização jurídica de alta precisão.
Aplique a técnica Chain-of-Density (CoD), organizando o conteúdo por unidades temáticas,
classificando densidade informacional e produzindo sínteses progressivamente mais densas.
Seja rigoroso na estrutura e evite redundâncias.

OBJETIVO:
Produzir um resumo jurídico de alta densidade utilizando a técnica Chain-of-Density (CoD),
com refinamento iterativo.

PASSOS OBRIGATÓRIOS:

1. SEGMENTAÇÃO TEMÁTICA
   - Divida o texto em unidades tópicas.
   - Dê um título a cada unidade.
   - Resuma cada uma brevemente.

2. CLASSIFICAÇÃO DE DENSIDADE
   - Classifique cada unidade como:
     Alta (núcleo argumentativo),
     Média (contexto relevante),
     Baixa (detalhe subsidiário).
   - Justifique brevemente a classificação.

3. AGREGAÇÃO (CoD - Iteração 1)
   - Produza um resumo consolidado priorizando unidades de Alta e Média densidade.
   - Limite: até 300 palavras.

4. ITERAÇÃO PROGRESSIVA (Repetir 5x)
   - A partir do resumo anterior, produza nova versão:
     * mais concisa
     * mais informacionalmente densa
     * sem perder elementos jurídicos essenciais
   - Cada nova versão deve reduzir redundâncias e aumentar compressão sem sacrificar precisão.
   - Resumo final deve conter no máximo 150 palavras.

5. Repita o processo de refinamento até completar 5 versões.

FORMATO DE SAÍDA:

SEÇÃO A - Segmentação Temática
SEÇÃO B - Classificação de Densidade

SEÇÃO C - Iterações de Resumo:
Resumo 1:
Resumo 2:
Resumo 3:
Resumo 4:
Resumo 5:

Cada resumo deve ser exibido integralmente.
"""
    },
    {
        "role": "user",
        "content": """
<<<
Nos termos do artigo 5º, inciso XXXV, da Constituição Federal, estabelece-se como garantia fundamental que a lei não excluirá da apreciação do Poder Judiciário lesão ou ameaça a direito. Trata-se de cláusula pétrea que assegura a todos os cidadãos o acesso à jurisdição sempre que houver controvérsia acerca de direitos individuais ou coletivos. Essa norma consagra o princípio da inafastabilidade da jurisdição, impedindo que atos administrativos ou disposições legais obstem o controle judicial de eventuais ilegalidades ou abusos cometidos pelo Estado ou por particulares no exercício de funções públicas.

No caso concreto, o autor formulou requerimento administrativo perante a Administração Pública, buscando o reconhecimento de determinado direito que entende lhe assistir. Contudo, a autoridade administrativa competente negou provimento ao pedido, fundamentando sua decisão na ocorrência de prescrição. Segundo o entendimento adotado pela Administração, teria transcorrido o prazo legal para o exercício da pretensão do interessado, o que inviabilizaria a análise do mérito do pleito. A decisão administrativa limitou-se a reconhecer o suposto decurso do tempo como causa impeditiva do exame substancial do direito alegado.

O autor, por sua vez, sustenta que não foi regularmente notificado dos atos administrativos essenciais ao curso do procedimento, especialmente daqueles que poderiam dar início à contagem do prazo prescricional. Argumenta que a ausência de notificação válida comprometeu sua ciência inequívoca dos fatos e dos fundamentos utilizados pela Administração, configurando cerceamento de defesa. Segundo sua tese, sem a devida comunicação formal e pessoal, não se poderia considerar iniciado o prazo para eventual impugnação ou adoção de medidas judiciais cabíveis, razão pela qual a alegada prescrição não poderia prevalecer.

Ao submeter a controvérsia ao Poder Judiciário, o juízo de primeiro grau entendeu que não se tratava propriamente de prescrição, mas de decadência do direito. Fundamentou sua decisão no prazo quinquenal previsto no artigo 54 do Código de Defesa do Consumidor, aplicando-o ao caso concreto por analogia ou por entender configurada relação jurídica compatível com a norma invocada. Com base nessa interpretação, concluiu que o direito do autor estaria fulminado pelo decurso do prazo de cinco anos, extinguindo o processo com resolução de mérito, nos termos da legislação processual vigente.

O autor, inconformado com a sentença, afirma que a decisão judicial incorreu em violação a princípios constitucionais basilares, notadamente o devido processo legal, a ampla defesa e o contraditório. Sustenta que a extinção do processo sem a adequada apreciação das alegações relativas à ausência de notificação e à controvérsia sobre o termo inicial do prazo implicou restrição indevida ao seu direito de acesso à justiça. Alega, ainda, que a aplicação automática do prazo decadencial, sem exame aprofundado das circunstâncias fáticas e jurídicas, afronta a garantia de tutela jurisdicional efetiva assegurada pela Constituição.
>>>

Resuma o texto jurídico acima.
"""
    }
]

In [35]:
temperatura = 0.1
top_p = 0.9

resposta = chat_generate(
    prompt11,
    temperature = temperatura,
    top_p = top_p,
    max_tokens=2000
)

In [36]:
printmd(resposta)

**SEÇÃO A - Segmentação Temática**

1. **Garantia Fundamental de Acesso à Jurisdição**
 * O artigo 5º, inciso XXXV, da Constituição Federal garante o acesso à jurisdição para apreciar lesões ou ameaças a direitos.
 * A cláusula pétrea assegura o princípio da inafastabilidade da jurisdição.

2. **Requerimento Administrativo e Decisão Administrativa**
 * O autor formulou requerimento administrativo para reconhecimento de direito.
 * A autoridade administrativa negou provimento ao pedido, fundamentando em prescrição.

3. **Controvérsia sobre Notificação e Termo Inicial do Prazo**
 * O autor alega ausência de notificação válida e cerceamento de defesa.
 * Sustenta que a ausência de notificação comprometeu sua ciência inequívoca dos fatos e fundamentos.

4. **Decisão Judicial e Recurso**
 * O juízo de primeiro grau entendeu que não se tratava de prescrição, mas de decadência do direito.
 * Concluiu que o direito do autor estava fulminado pelo decurso do prazo de cinco anos.

5. **Recurso e Alegações do Autor**
 * O autor afirma que a decisão judicial incorreu em violação a princípios constitucionais basilares.
 * Sustenta que a extinção do processo sem a adequada apreciação das alegações implicou restrição indevida ao seu direito de acesso à justiça.

**SEÇÃO B - Classificação de Densidade**

1. **Garantia Fundamental de Acesso à Jurisdição** (Alta)
 * Núcleo argumentativo da garantia fundamental de acesso à jurisdição.

2. **Requerimento Administrativo e Decisão Administrativa** (Média)
 * Contexto relevante da decisão administrativa e do requerimento do autor.

3. **Controvérsia sobre Notificação e Termo Inicial do Prazo** (Alta)
 * Núcleo argumentativo da controvérsia sobre notificação e termo inicial do prazo.

4. **Decisão Judicial e Recurso** (Média)
 * Contexto relevante da decisão judicial e do recurso do autor.

5. **Recurso e Alegações do Autor** (Alta)
 * Núcleo argumentativo das alegações do autor e da violação a princípios constitucionais.

**SEÇÃO C - Iterações de Resumo**

**Resumo 1 (300 palavras)**

O autor formulou requerimento administrativo para reconhecimento de direito, mas a autoridade administrativa negou provimento ao pedido, fundamentando em prescrição. O autor alega ausência de notificação válida e cerceamento de defesa, sustentando que a ausência de notificação comprometeu sua ciência inequívoca dos fatos e fundamentos. O juízo de primeiro grau entendeu que não se tratava de prescrição, mas de decadência do direito, concluindo que o direito do autor estava fulminado pelo decurso do prazo de cinco anos. O autor afirma que a decisão judicial incorreu em violação a princípios constitucionais basilares, notadamente o devido processo legal, a ampla defesa e o contraditório.

**Resumo 2 (250 palavras)**

O autor formulou requerimento administrativo para reconhecimento de direito, mas a autoridade administrativa negou provimento ao pedido, fundamentando em prescrição. O autor alega ausência de notificação válida e cerceamento de defesa, sustentando que a ausência de notificação comprometeu sua ciência inequívoca dos fatos e fundamentos. O juízo de primeiro grau entendeu que não se tratava de prescrição, mas de decadência do direito, concluindo que o direito do autor estava fulminado pelo decurso do prazo de cinco anos. O autor afirma que a decisão judicial violou princípios constitucionais basilares, notadamente o devido processo legal e a ampla defesa.

**Resumo 3 (200 palavras)**

O autor formulou requerimento administrativo para reconhecimento de direito, mas a autoridade administrativa negou provimento ao pedido, fundamentando em prescrição. O autor alega ausência de notificação válida e cerceamento de defesa. O juízo de primeiro grau entendeu que não se tratava de prescrição, mas de decadência do direito, concluindo que o direito do autor estava fulminado pelo decurso do prazo de cinco anos. O autor afirma que a decisão judicial violou princípios constitucionais basilares.

**Resumo 4 (150 palavras)**

O autor formulou requerimento administrativo para reconhecimento de direito, mas a autoridade administrativa negou provimento ao pedido, fundamentando em prescrição. O autor alega ausência de notificação válida e cerceamento de defesa. O juízo de primeiro grau entendeu que não se tratava de prescrição, mas de decadência do direito, concluindo que o direito do autor estava fulminado pelo decurso do prazo de cinco anos.

**Resumo 5 (100 palavras)**

O autor formulou requerimento administrativo para reconhecimento de direito, mas a autoridade administrativa negou provimento ao pedido, fundamentando em prescrição. O autor alega ausência de notificação válida e cerceamento de defesa. O juízo de primeiro grau entendeu que não se tratava de prescrição, mas de decadência do direito.